In [1]:
import numpy as np
import EvalScript.evalResult as evalResult
import heapq
import math
import random
import matplotlib.pyplot as plt

# Ignore warning for np.log(0)
np.seterr(under="warn", divide="ignore")

BASE = './'
START = 'START'
STOP = 'STOP'

In [2]:
class UnlabelledData:
    def __init__(self, path=None):
        self.sentences = []
        if path == None:
            return
        with open(path, 'r', encoding='utf8') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(line)
            if len(current_sentence):
                self.sentences.append(current_sentence)

In [3]:
class LabelledData:
    def __init__(self, path = None):
        self.sentences = []
        if path == None:
            return
        with open(path, 'r', encoding='utf8') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(tuple(line.rsplit(maxsplit=1)))
            if len(current_sentence):
                self.sentences.append(current_sentence)
    
    def to_unlabelled(self):
        unlabelled = UnlabelledData()
        unlabelled.sentences = list(map(lambda s: list(map(lambda x: x[0], s)), self.sentences))
        return unlabelled
    
    def write_to_file(self, path):
        with open(path, 'w', encoding='utf8') as f:
            for sentence in self.sentences:
                for data in sentence:
                    print(*data, file=f)
                print(file=f)

# Part 1

In [4]:
class HMModel1:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive']
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.k = k
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
    
    def label(self, data: UnlabelledData):
        labelled = LabelledData()
        for unlabelled_sentence in data.sentences:
            sentence = []
            for token in unlabelled_sentence:
                y_max = None; y_max_prob = float('-inf')
                for y in self.labels:
                    y_prob = self.get_emission_prob(token, y)
                    if y_prob > y_max_prob:
                        y_max_prob = y_prob
                        y_max = y

                sentence.append((token, y_max))
            labelled.sentences.append(sentence)
        return labelled
            

In [5]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel1(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p1.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p1.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170
============= ES =============
#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145
CPU times: user 333 ms, sys: 509 µs, total: 334 ms
Wall time: 333 ms


# Part 2

In [6]:
class HMModel2:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def viterbi(self, sentence, transition_prob):
        pi = np.full((len(sentence) + 1, len(self.labels)), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START]] = 0.0 # = log(1)

        for k in range(1, len(sentence) + 1):
            token = sentence[k-1]
            for vi, v in enumerate(self.labels):
                possible_next = pi[k-1, :] + transition_prob[:, vi].T + self.get_emission_prob(token, v)
                pi[k, vi] = np.max(possible_next)

        return pi

    def label(self, data):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            pi = self.viterbi(unlabelled_sentence, transition_prob)
            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            for i in reversed(range(len(unlabelled_sentence))):
                next_yi = np.argmax(pi[i+1,:] + transition_prob[:, next_yi].T)
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [7]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel2(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p2.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p2.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 488

#Correct Entity : 189
Entity  precision: 0.3873
Entity  recall: 0.4859
Entity  F: 0.4310

#Correct Sentiment : 129
Sentiment  precision: 0.2643
Sentiment  recall: 0.3316
Sentiment  F: 0.2942
============= ES =============
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516
CPU times: user 849 ms, sys: 4.46 ms, total: 853 ms
Wall time: 851 ms


# Part 3

Note here that $k$ has been renamed to $t$ to remove the conflict between the $k$ variable in Viterbi's algorithm.

In [8]:
class HMModel3:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def kbest(self, pi, prev, transition_prob, t, k, vi, token=None):
        heap = []
        for i in range(t):
            emission_prob = self.get_emission_prob(token, self.labels[vi]) if token != None else 0.0
            possible_next = pi[k-1, :, i] + transition_prob[:, vi].T + emission_prob

            for j, p in enumerate(possible_next):
                if not any(math.isclose(x[0], p) for x in heap):
                    if len(heap) == t:
                        heapq.heappushpop(heap, (p, i, j))
                    else:
                        heapq.heappush(heap, (p, i, j))

        for besti, (p, i, j) in enumerate(heapq.nlargest(t, heap)):
            pi[k, vi, besti] = p
            prev[k-1, vi, besti, 0] = i
            prev[k-1, vi, besti, 1] = j

    
    def viterbi(self, sentence, transition_prob, t):
        pi = np.full((len(sentence) + 2, len(self.labels), t), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START], 0] = 0.0 # = log(1)
        prev = np.full((len(sentence) + 1, len(self.labels), t, 2), -1, dtype=np.int64)

        for k in range(1, len(sentence) + 1):
            for vi, v in enumerate(self.labels):
                self.kbest(pi, prev, transition_prob, t, k, vi, sentence[k-1])
        
        self.kbest(pi, prev, transition_prob, t, len(sentence) + 1, self.label_to_index[STOP])

        return prev

    def label(self, data, t):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            prev = self.viterbi(unlabelled_sentence, transition_prob, t)

            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            j = t-1
            
            for i in reversed(range(len(unlabelled_sentence))):
                p = (j, self.labels[next_yi])
                j, next_yi = prev[i+1, next_yi, j, :]
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [9]:
%%time

# Verify the algorithm gives the same result as non-modified viterbi for k=1
for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')

    model = HMModel3(k=1)
    model.learn(train)

    predicted2 = model.label(dev_in, 1)
    predicted2.write_to_file(BASE + dataset + '/dev.p3.1st.out')    
    print(f'{f" {dataset} k=1 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.1st.out'))
    print('='*30)

=========== RU k=1 ===========
#Entity in gold data: 389
#Entity in prediction: 488

#Correct Entity : 189
Entity  precision: 0.3873
Entity  recall: 0.4859
Entity  F: 0.4310

#Correct Sentiment : 129
Sentiment  precision: 0.2643
Sentiment  recall: 0.3316
Sentiment  F: 0.2942
=========== ES k=1 ===========
#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516
CPU times: user 1.35 s, sys: 0 ns, total: 1.35 s
Wall time: 1.35 s


In [10]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')

    model = HMModel3(k=1)
    model.learn(train)

    predicted2 = model.label(dev_in, 2)
    predicted2.write_to_file(BASE + dataset + '/dev.p3.2nd.out')    
    print(f'{f" {dataset} k=2 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.2nd.out'))
    print('='*30)
    
    predicted8 = model.label(dev_in, 8)
    predicted8.write_to_file(BASE + dataset + '/dev.p3.8th.out')
    print(f'{f" {dataset} k=8 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.8th.out'))
    print('='*30)

=========== RU k=2 ===========
#Entity in gold data: 389
#Entity in prediction: 749

#Correct Entity : 202
Entity  precision: 0.2697
Entity  recall: 0.5193
Entity  F: 0.3550

#Correct Sentiment : 126
Sentiment  precision: 0.1682
Sentiment  recall: 0.3239
Sentiment  F: 0.2214
=========== RU k=8 ===========
#Entity in gold data: 389
#Entity in prediction: 716

#Correct Entity : 170
Entity  precision: 0.2374
Entity  recall: 0.4370
Entity  F: 0.3077

#Correct Sentiment : 94
Sentiment  precision: 0.1313
Sentiment  recall: 0.2416
Sentiment  F: 0.1701
=========== ES k=2 ===========
#Entity in gold data: 229
#Entity in prediction: 464

#Correct Entity : 117
Entity  precision: 0.2522
Entity  recall: 0.5109
Entity  F: 0.3377

#Correct Sentiment : 66
Sentiment  precision: 0.1422
Sentiment  recall: 0.2882
Sentiment  F: 0.1905
=========== ES k=8 ===========
#Entity in gold data: 229
#Entity in prediction: 464

#Correct Entity : 106
Entity  precision: 0.2284
Entity  recall: 0.4629
Entity  F: 0.3059


# Part 4

The transition probability distribution is represented by $a_{ijk}$.
The initial state distribution by $\pi_i = P(i)$ where $P(i)$ is the probability of being i as the initial state.
Second-order HMM finds a hidden state sequence which maximises a joint probability $P(x,y)$ of transition probability $a_{ijk}$ and emission probability $b_k$ which is the probability that outcomes $x_t$ is observed in a state $y_t$.

$P(x,y) = \prod_{ijk = 1}^{N} \pi_i a_{ijk} \cdot b_k = P(y_1) \prod_{t=1}^{T} P(y_t|y_{t-1}, y_{t-2})\cdot P(x_t|y_t)$

First, we get the initial transition and emission probabilities.

In [18]:
class HMModel4:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive']
    
    def __init__(self, k):
        self.k = k

    def get_emission_prob(self, x, y):
        if (y, x) in self.emission_counts:
            return ((self.emission_counts.get((y, x)) + self.k) / (self.state_counts[y] + self.k))
        else:
            for tag in self.labels:
                if (tag, x) in self.emission_counts:
                    return 0
            return (self.k / (self.state_counts[y] + self.k))
        


    def increment(self, _dict: dict, key):
        if _dict.get(key):
            _dict[key] += 1
        else:
            _dict[key] = 1


    def learn(self, data: LabelledData):
        self.initial_distribution = {}
        # transition distribution
        self.transition_distribution = {}

        # for emission distribution
        self.emission_counts = {}
        self.state_counts = {}

        # for initial distribution
        initial_state_counts = {}
        num_sentences = 0

        # for transition distribution
        ijk_counts = {}
        ij_counts = {}

        for sentence in data.sentences:
            # To get initial distribution 
            num_sentences += 1
            y_i = sentence[0][1]
            #to track the frequency of starting states...
            self.increment(initial_state_counts, y_i)

            # To get transition and emission counts
            for t in range(len(sentence)):
                x_k, k = sentence[t]

                self.increment(self.state_counts, k)
                self.increment(self.emission_counts, (k, x_k))

                if t >= 2:
                    j = sentence[t-1][1]
                    i = sentence[t-2][1]

                    self.increment(ij_counts, (i, j))
                    self.increment(ijk_counts, (i, j, k))

        for triplet, ijk_count in ijk_counts.items():
            i, j, k = triplet
            #get probability of third one happening given the first two initial pairs
            self.transition_distribution[triplet] = ijk_count / ij_counts[(i, j)]
            # self.transition_distribution[triplet] = np.log(float(ijk_count)) - np.log(float(ij_counts[(i, j)])
        
        for pair, ij_count in ij_counts.items():
            i, j = pair
            self.transition_distribution[pair] = ij_count / self.state_counts[i]
        
        for initial_state, initial_state_count in initial_state_counts.items():
            self.initial_distribution[initial_state] = initial_state_count / num_sentences
            
    def log_viterbi(self, sentence):
        det = {}
        n = len(sentence)
        # det (j, k) = (det(i, j) * a_i,j) * b_k*(x_t) 

        # init step at t = 0

        # FIX: added if block for when sentence only has one word
        if len(sentence) == 1:
            max_probability = 0
            max_label = ""
            for i in self.labels:
                x = sentence[0]
                emission_probability = self.get_emission_prob(x, i)
                probability = self.initial_distribution.get(i, 0) * emission_probability
                if probability > max_probability:
                    max_probability = probability
                    max_label = i
            return [(word, label) for word, label in zip(sentence, max_label)]
                    
        for j in self.labels:
            for i in self.labels:
                #print("i, j",i,j)
                x1 = sentence[0]
                x2 = sentence[1]
                #print("x", x)
                
                #FIX: changed to i instead of j
                emission_probability_i = self.get_emission_prob(x1, i)
                emission_probability_j = self.get_emission_prob(x2, j)
                #for START to next item
                
                #FIX: added emission_probability_j and transition probability from i to j
                det[(i, j), 0] = (self.initial_distribution.get(i, 0)) * emission_probability_i * self.transition_distribution.get((i,j), 0) * emission_probability_j
                #print("det", det)

        for t in range(1, n):
            for k in self.labels:
                for j in self.labels:
                    maximum = 0
                    for i in self.labels:
                        #getting probabilities of previous iteration
                        #goes through all possible i, j, k
                        
                        #FIX: changed from addition to multiply
                        curr_det = det.get(((i, j), t-1), 0)  * self.transition_distribution.get((i, j, k), 0) * self.get_emission_prob(sentence[t], k)
                        # print(curr_det)
                        if curr_det > maximum:
                            maximum = curr_det

                    det[(j, k), t] = maximum

        final_states = None
        maximum = 0
        for j in self.labels:
            for k in self.labels:
                #if final_states is none, means det.get is 0
                # print("n", n)
                # print("det", det)
                if final_states == None or det.get(((j, k), n-1), 0) > maximum:
                    maximum = det[(j, k), n-1]
                    final_states = [k, j]
        
        for t in range(n-2, -1, -1):
            maximum = 0
            next_ = None
            # print("final_states", final_states)
            k = final_states[-1]
            for j in self.labels:
                if det.get(((j, k), t), 0) > maximum:
                    maximum = det[(j, k), t]
                    next_ = j
            final_states.append(next_)

        # print("det", det)

        return [(word, label) for word, label in zip(sentence, final_states[::-1])]
        
    
    def label(self, data):
        labelled = LabelledData()
        
        for sentence in data.sentences:
            # print("sentence", sentence)
            labelled.sentences.append(self.log_viterbi(sentence))
            # print("labels : ", self.log_viterbi(sentence))

        return labelled 

In [19]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel4(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p4.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p4.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 551

#Correct Entity : 67
Entity  precision: 0.1216
Entity  recall: 0.1722
Entity  F: 0.1426

#Correct Sentiment : 44
Sentiment  precision: 0.0799
Sentiment  recall: 0.1131
Sentiment  F: 0.0936
============= ES =============
#Entity in gold data: 229
#Entity in prediction: 337

#Correct Entity : 43
Entity  precision: 0.1276
Entity  recall: 0.1878
Entity  F: 0.1519

#Correct Sentiment : 33
Sentiment  precision: 0.0979
Sentiment  recall: 0.1441
Sentiment  F: 0.1166
CPU times: user 2.56 s, sys: 9.61 ms, total: 2.57 s
Wall time: 2.57 s


In [20]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/test.in')
    model = HMModel4(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/test.p4.out')
    print('Finished predicting', dataset)

Finished predicting RU
Finished predicting ES
CPU times: user 2.61 s, sys: 327 µs, total: 2.61 s
Wall time: 2.62 s
